# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword = "網路爬蟲"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

網路爬蟲: %E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2
/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [7]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


網路爬蟲（英語：web crawler），也叫網路蜘蛛（spider），是一種用來自動瀏覽全球資訊網的網路機器人（英語：Internet bot）。其目的一般為編纂網路索引（英語：Web indexing）。

網路搜尋引擎等站點通過爬蟲軟體更新自身的網站內容（英語：Web content）或其對其他網站的索引。網路爬蟲可以將自己所存取的頁面儲存下來，以便搜尋引擎事後生成索引（英語：Index (search engine)）供用戶搜尋。

爬蟲存取網站的過程會消耗目標系統資源。不少網路系統並不默許爬蟲工作。因此在存取大量頁面時，爬蟲需要考慮到規劃、負載，還需要講「禮貌」。 不願意被爬蟲存取、被爬蟲主人知曉的公開站點可以使用robots.txt檔案之類的方法避免存取。這個檔案可以要求機器人（英語：Software agent）只對網站的一部分進行索引，或完全不作處理。

網際網路上的頁面極多，即使是最大的爬蟲系統也無法做出完整的索引。因此在公元2000年之前的全球資訊網出現初期，搜尋引擎經常找不到多少相關結果。現在的搜尋引擎在這方面已經進步很多，能夠即刻給出高品質結果。

爬蟲還可以驗證超連結和HTML程式碼，用於網路抓取（英語：Web scraping）（參見資料驅動編程（英語：Data-driven programming））。

網路爬蟲也可稱作網路蜘蛛[1]、螞蟻、自動索引程式（automatic indexer）[2] ，或（在FOAF（英語：FOAF (software)）軟體中）稱為網路疾走（web scutter）。[3]

網路爬蟲始於一張被稱作種子的統一資源位址（URL）列表。當網路爬蟲存取這些統一資源定位器時，它們會甄別出頁面上所有的超連結，並將它們寫入一張「待訪列表」，即所謂爬行疆域（英語：crawl frontier）。此疆域上的URL將會被按照一套策略迴圈來存取。如果爬蟲在執行的過程中複製歸檔和儲存網站上的資訊，這些檔案通常儲存，使他們可以較容易的被檢視。閱讀和瀏覽他們儲存的網站上並即時更新的資訊，這些被儲存的網頁又被稱為「快照」。越大容量的網頁意味著網路爬蟲只能在給予的時間內下載越少部分的網頁，所以要優先考慮其下載。高變化率意味著網頁可能已經被更新或者被取代。一些伺服器端軟體生成的URL（統一資源定位符）也使得網路爬蟲很難避免檢索到重複

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [11]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [全球資訊網] /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
外部連結: [網路] /wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: [搜尋引擎] /wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: [robots.txt] /wiki/Robots.txt
外部連結: [網站] /wiki/%E7%BD%91%E7%AB%99
外部連結: [超連結] /wiki/%E8%B6%85%E9%80%A3%E7%B5%90
外部連結: [HTML] /wiki/HTML
外部連結: [網頁] /wiki/%E7%B6%B2%E9%A0%81
外部連結: [網際網路] /wiki/%E4%BA%92%E8%81%94%E7%BD%91
外部連結: [伺服器] /wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8
外部連結: [超文字傳輸協定] /wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [3]:
URL = 'https://zh.wikipedia.org/wiki/Python'
resp = requests.get(url=URL)

In [22]:
def spider(key):
    keyword = []
    article = []
    content = []
    n = ''
    URL = 'https://zh.wikipedia.org/wiki/'.format(key)
    resp = requests.get(url=URL)
    soup = BeautifulSoup(markup=resp.text, features='html.parser')
    nodes = soup.select(selector = 'p')
    for i in nodes:
        n += i.text.strip()
    content.append(n)
    
    nodes2 = soup.select(selector = 'p a')
    for j in nodes2:
        keyword.append(j.text)
    
    for n in keyword:
        URL = 'https://zh.wikipedia.org/wiki/{}'.format(n)
        resp = requests.get(url = URL)
        soup = BeautifulSoup(markup=resp.text, features='html.parser')
        nodes3 = soup.select(selector = 'p')
        n1 = ''
        for x in nodes3:
            n1 += x.text.strip()
        article.append(n1)
    return content, keyword, article

In [23]:
test = spider('志工')

In [34]:
print(test[0])

['海納百川，有容乃大人人可編輯的自由百科全書已有1,198,819篇條目1706年5月23日爆发的拉米伊战役，是西班牙王位继承战争期间的一场战役。对于大同盟成员奥地利、英格兰与荷兰共和国而言，这场战役是在1705年对法国国王路易十四的波旁军队展开的一系列非决定性军事行动之后发生的。在前期胜利的鼓舞下，法国元帅率领60000大军从鲁汶出征，向蒂嫩进发。英荷联军总司令马尔伯勒公爵同样决心大干一场，他在马斯特里赫特附近集结了约62000人的部队。在不到四个小时的时间里，马尔伯勒率领的荷兰、英格兰和丹麦部队便打败了维勒鲁瓦和马克斯·埃马努埃尔率领的法国-西班牙-巴伐利亚联军。最终维勒鲁瓦的军队被赶出西属尼德兰的大部分地区。西风带是中纬度地区自西向东吹的盛行风，由副热带高压带的高压区域吹向极地区域。北半球的西风带主要吹西南风，南半球则主要吹西北风。当半球处于冬季且极地附近的气压较低时，西风带最强；而当半球处于夏季且极地附近的气压较高时，西风带最弱。南半球的中纬度地区少有会导致西风减速的陆地，因此盛行西风格外强。西风带中存在着各类温带气旋和反气旋。由于西风的增加，越过副热带高压脊线进入西风带的热带气旋会转向，并可能发生温带变性。盛行西风还把温暖的赤道水和空气带到中纬度大陆的西海岸，对其气候特征起着重要作用。用鸡肉、辣椒、南瓜、豌豆和番茄做的米饭。5月24日：厄立特里亚獨立日（1993年）维基百科是一个多语言、内容自由、任何人都能参与的协作计划，其目标是建立一个完整、准确且中立的百科全书。中文维基百科的成长依靠您的参与，無論是创建新条目、编辑现有条目，或者为条目增加插图，您都可以为维基百科作出贡献。重定向：是一種特殊的頁面，用途在於讓進入者自動被導入另外一個頁面。维基百科由非营利组织维基媒体基金会运作。基金会旗下尚有其他数个多语言、内容开放的维基计划：']


In [35]:
print(test[1])

['人人可編輯', '自由', '1,198,819', '條目', '拉米伊战役', '西班牙王位继承战争', '战役', '大同盟', '法国国王', '路易十四', '波旁', '鲁汶', '蒂嫩', '马尔伯勒公爵', '马斯特里赫特', '马克斯·埃马努埃尔', '西风带', '中纬度', '盛行风', '副热带高压带', '极地', '风', '冬季', '夏季', '温带气旋', '反气旋', '副热带高压脊线', '热带气旋', '温带变性', '赤道', '大陆', '5月24日', '厄立特里亚', '獨立日', '1993年', '维基百科', '多语言', '内容自由', '任何人都能参与', '创建新条目', '编辑现有条目', '为条目增加插图', '重定向', '维基媒体基金会']


In [37]:
print(test[2][0:3])

['维基百科目前还没有与上述标题相同的Article。请先在维基百科上查找“人人可編輯”是否已有名称相近或不同文字写法的条目。可能出现此提示的其他原因：', '自由（和製漢語：自由，英語：Freedom，Liberty）是一個哲學中的概念，意即可以自我支配，凭借意志而行动。學術上存在對自由的概念有不同見解，在对个人与社会的關係认识上有所不同。自由包括各国宪法规定的言论信息自由和新闻自由，當然還有諸多的自由，例如：思想自由、宗教信仰自由等等。法律概念上則以憲法來保障人的自由權，並同時賦予其國民守護享有各該憲法保障之各種自由權利之義務。自由，在民主政治中又体现为一种精神：就是对什么是正确不那么确定，但是會守護那有選擇之自由的精神。自由(英語：Freedom，Liberty)有多种含义：而法国大革命纲领性文件《人权宣言》中，对自由的定义为：絕對的自由在理論上可能存在，但由於社會是由人與人所組成，自由不僅是個人的議題，而是社會中各個主體之間彼此互相界定的程度，因此湯瑪斯·傑弗遜認為个人的自由受制于他人的同等的自由。進而有義務論者和馬克思主義者認為自由與責任相關，有相關之自由即應負相關之責任。自由的边界是人权，自由止于人权。自由是政治哲学的核心概念。自由也是一种社会概念。自由是社会人的权利。与自由相对的，是奴役。孫中山多次在演講中引述彌爾的話指出，一個人的自由，以不侵犯他人的自由為範圍，才是真自由。如果侵犯他人的範圍，便是不自由[2]。第二次世界大战中（1941年1月6日），美国总统罗斯福在国情咨文中提出了著名的“四大自由”[3]。在未来的日子里，我们向往并需要确保一个由必要的人类自由构成的世界：  第一，言论与表达的自由——在世界上的所有地方。  第二，一切人民以自己的方式崇拜他们神祇的自由——在世界上的所有地方。  第三，免于匮乏的自由——以世界性的角度来说，可使每一个国家的所有居民享受在和平时期健康生活的经济基础——在世界上的所有地方。  第四，免于恐惧的自由——以世界性的角度来说，全球的兵力削减以至于任何国家都没有对其邻邦进行武力入侵的可能——在世界上的任何地方。简略来说，这四大自由便是言论自由、宗教自由、免于匮乏的自由和免于恐惧的自由。联合国世界人权宣言重申了这四大自由的精神：「人人享有言論和信仰自由並免于恐懼和匱乏」(《世界人權宣言》)。20世纪下半叶，以賽亞·伯